In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day < 365 THEN 'new'
           ELSE 'old'
       END AS is_new,
       u.age,
       CASE
           WHEN u.gender_segment = 1 THEN 'women'
           WHEN u.gender_segment = 0 THEN 'men'
           ELSE 'n/a'
       END gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       substr(ages.title, -5, 5) AS age_segment,
       ts.title AS traffic_segment,
       ls.title AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >=9 THEN 'promoters'
           WHEN u.nps_score >=7 THEN 'passives'
           ELSE 'detractors'
       END nps_group
FROM USER AS u
JOIN LOCATION AS l ON l. location_id = u.location_id
JOIN age_segment AS ages ON ages.age_gr_id = u.age_gr_id
JOIN traffic_segment AS ts ON ts.tr_gr_id = u.tr_gr_id
JOIN lifetime_segment AS ls ON ls.lt_gr_id = u.lt_gr_id

"""

In [4]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,women,ANDROID,SMARTPHONE,Россия,Уфа,45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,53.0,men,ANDROID,SMARTPHONE,Россия,Киров,45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,57.0,men,ANDROID,SMARTPHONE,Россия,Москва,55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,old,44.0,women,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,old,24.0,men,ANDROID,SMARTPHONE,Россия,Рязань,16-24,05 5-10,08 36+,10,promoters


In [5]:
df['gender_segment'].unique()

array(['women', 'men', 'n/a'], dtype=object)

In [6]:
gender = df.pivot_table(index='gender_segment', values='user_id', aggfunc=['count']) \
                                .sort_values(by=([('count', 'user_id')]), ascending=False)
gender = gender.reset_index()
gender.columns = ['gender', 'count']
gender['%'] = (100 * gender['count']/gender['count'].sum()).round(1)
gender

,gender,count,%
0,women,272442,54.2
1,men,228750,45.5
2,n/a,1301,0.3


In [7]:
os_name = df.pivot_table(index='os_name', values='user_id', aggfunc=['count']) \
                                .sort_values(by=([('count', 'user_id')]), ascending=False)
os_name = os_name.reset_index()
os_name.columns = ['os_name', 'count']
os_name['%'] = (100 * os_name['count']/os_name['count'].sum()).round(1)
os_name

,os_name,count,%
0,ANDROID,435981,86.8
1,IOS,60051,12.0
2,PROPRIETARY,2491,0.5
3,OTHER,2394,0.5
4,WINDOWS PHONE,1297,0.3
5,unknown,133,0.0
6,SYMBIAN OS,103,0.0
7,BADA OS,40,0.0
8,WINDOWS MOBILE,3,0.0


In [8]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

[Дашборд](https://public.tableau.com/views/_NPS_Final/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

[Презентация](https://disk.yandex.ru/i/Yc_gVE-Cu0OfDw)